In [1]:
import torch
from torch import nn,save,load
import torch.nn.functional as f
from torch.optim import SGD,Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
tr=datasets.MNIST(root='data',download=True,train=True,transform=ToTensor())
dataset=DataLoader(tr,32) # batch size 32

class Imageclass(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6),10)
    )

  def forward(self,x):
    return self.model(x)

clf=Imageclass().to('cuda')
opt=Adam(clf.parameters(),lr=1e-3)
loss_fn=nn.CrossEntropyLoss()

if __name__ == '__main__':
  for epoch in range(20):
    for batch in dataset:
      X,y=batch
      X,y=X.to('cuda'),y.to('cuda')
      yhat=clf(X)
      loss=loss_fn(yhat,y)

      opt.zero_grad()
      loss.backward()
      opt.step()

    print(f'epoch #{epoch} loss -> {loss.item()}')



epoch #0 loss -> 0.032445646822452545
epoch #1 loss -> 0.0037373702507466078
epoch #2 loss -> 0.009773761965334415
epoch #3 loss -> 4.5782613597111776e-05
epoch #4 loss -> 0.0015909241046756506
epoch #5 loss -> 0.00010299100540578365
epoch #6 loss -> 1.0094017852679826e-05
epoch #7 loss -> 0.029039189219474792
epoch #8 loss -> 2.4959362576737476e-07
epoch #9 loss -> 8.94068392653935e-08
epoch #10 loss -> 2.451157115501701e-06
epoch #11 loss -> 1.4901159417490817e-08
epoch #12 loss -> 2.6449492906976957e-07
epoch #13 loss -> 5.960459503739912e-08
epoch #14 loss -> 2.495937678759219e-07
epoch #15 loss -> 2.6077021431092362e-08
epoch #16 loss -> 0.0
epoch #17 loss -> 2.9802308176840597e-08
epoch #18 loss -> 4.917357045997051e-07
epoch #19 loss -> 0.0


In [7]:
with open('model_stat.pt','wb') as f:
  save(clf.state_dict(),f)

In [9]:
import torch
from PIL import Image

In [13]:
!ls

data  img_1.jpg  img_2.jpg  img_3.jpg  model_stat.pt  sample_data


In [14]:
with open('model_stat.pt','rb') as f:
  clf.load_state_dict(load(f))
  img =Image.open('img_1.jpg')
  img_ten=ToTensor()(img).unsqueeze(0).to('cuda')
  print(torch.argmax(clf(img_ten)))

tensor(2, device='cuda:0')


In [15]:
with open('model_stat.pt','rb') as f:
  clf.load_state_dict(load(f))
  img =Image.open('img_3.jpg')
  img_ten=ToTensor()(img).unsqueeze(0).to('cuda')
  print(torch.argmax(clf(img_ten)))

tensor(9, device='cuda:0')


In [16]:
with open('model_stat.pt','rb') as f:
  clf.load_state_dict(load(f))
  img =Image.open('img_2.jpg')
  img_ten=ToTensor()(img).unsqueeze(0).to('cuda')
  print(torch.argmax(clf(img_ten)))

tensor(0, device='cuda:0')
